In [1]:
import tensorflow as tf

In [2]:
#파일에서 데이터 로드
title_arr = []
ctr_arr = []
with open('../data/title_ctr.csv','r') as f:
    while True:
        line = f.readline()
        if not line:
            break;
        title_info = line.split(',')
        title = title_info[2]
        if len(title) > 0:
            title_arr.append(title)
            ctr_arr.append(float(title_info[3].replace('\n','')))
        
print(title_arr[:10])
print(ctr_arr[:10])

['초강력 워터프루프', '셔츠 초간단 연출법', '센스 넘치는 여행 소품들', '부위별 셀프 운동법', '이국주 퍼스널리티 오전', '강동원 인형놀이 오후', '호위무사 박수경 팬클럽까지 예쁘면 모든게 용서되나 경악 저녁', '빈 라덴 사살됐다 살아있다 여전히 풀리지 않는 미스터리 저녁', '남친에게 이런 짓까지 미녀스타의 충격적 실체 저녁', '택배등기 반송메시지 조심해야 하는 이유 저녁']
[0.0211, 0.0144, 0.0091, 0.0209, 0.0177, 0.0091, 0.0338, 0.0142, 0.0901, 0.0148]


In [3]:
#명사 추출
from konlpy.tag import Twitter
twitter_nlp = Twitter()

title_noun_arr = []
for index, title in enumerate(title_arr):
    if index % 10000 == 0:
        print('step:',index)
    title_noun_arr.append(twitter_nlp.nouns(title)) #명사 추출
print(title_noun_arr[:10])


step: 0
step: 10000
step: 20000
step: 30000
step: 40000
step: 50000
[['초', '강력', '워터', '프루프'], ['셔츠', '초', '간단', '연출', '법'], ['센스', '여행', '소품'], ['부위', '별', '셀프', '운동', '법'], ['이국주', '퍼스', '널리', '티', '오전'], ['강동원', '인형', '오후'], ['호위', '무사', '박수경', '팬클럽', '모든', '용서', '경악', '저녁'], ['빈', '라덴', '사살', '미스터리', '저녁'], ['남친', '짓', '미녀', '스타', '충격', '실체', '저녁'], ['택배', '등기', '반송', '메시지', '이유', '저녁']]


In [4]:
#벡터화 using word2vec
import gensim
import numpy as np

MIN_NOUN_VECTOR_VALUE = -10.0
MAX_NOUN_VECTOR_VALUE = 10.0
NOUN_VECTOR_SIZE = 300

def generate_random_noun_vector():
    return np.random.uniform(low=MIN_NOUN_VECTOR_VALUE, high=MAX_NOUN_VECTOR_VALUE, size=(NOUN_VECTOR_SIZE,))

w2v_model = gensim.models.Word2Vec.load('../data/w2v_model_wiki_kor')
w2v_model['남자']

title_noun_vector_arr = []
for index, title_nouns in enumerate(title_noun_arr):
    if index % 10000 == 0:
        print('step:',index)
    noun_vector_arr = []
    for noun in title_nouns:
        try:
            noun_vector = w2v_model[noun]
        except Exception as e:
            noun_vector = generate_random_noun_vector()
        noun_vector_arr.append(noun_vector)
    title_noun_vector_arr.append(noun_vector_arr)

# print(title_noun_vector_arr[:3])

step: 0
step: 10000
step: 20000
step: 30000
step: 40000
step: 50000


In [5]:
#pedding

TITLE_LENGTH = 25

def generate_zero_noun_vector():
    return np.random.uniform(low=0.0, high=0.0, size=(NOUN_VECTOR_SIZE,))

for index, title_noun_vector in enumerate(title_noun_vector_arr):
    if index % 10000 == 0:
        print('step:',index)
    while len(title_noun_vector) < 25:
        title_noun_vector.append(generate_zero_noun_vector())
    title_noun_vector = title_noun_vector[:25]

step: 0
step: 10000
step: 20000
step: 30000
step: 40000
step: 50000


In [6]:
#ctr to classification
NUM_CLASSES = 3
ctr_class_arr = []
ctr_class_count = [0,0,0]
for index, ctr in enumerate(ctr_arr):
    if index % 10000 == 0:
        print('step:',index)
    if ctr < 0.007:
        ctr_class_arr.append([0,0,1])
        ctr_class_count[2] += 1
    elif ctr < 0.012:
        ctr_class_arr.append([0,1,0])
        ctr_class_count[1] += 1
    else:
        ctr_class_arr.append([1,0,0])
        ctr_class_count[0] += 1

print(ctr_class_count)
print(ctr_class_arr[:5])

step: 0
step: 10000
step: 20000
step: 30000
step: 40000
step: 50000
[17005, 16347, 18116]
[[1, 0, 0], [1, 0, 0], [0, 1, 0], [1, 0, 0], [1, 0, 0]]


In [7]:
filter_sizes = [3,4,5]
embedding_size = 300
num_filters =100
l2_reg_lambda = 0.0
learning_rate = 1e-3

input_x = tf.placeholder(tf.float32, [None, TITLE_LENGTH, embedding_size], name="input_x")
input_y = tf.placeholder(tf.float32, [None, NUM_CLASSES], name="input_y")
input_x_expanded = tf.expand_dims(input_x, -1)
l2_loss = tf.constant(0.0)

pooled_outputs = []
for i, filter_size in enumerate(filter_sizes):
    with tf.name_scope("conv-maxpool-%s" % filter_size):
    
        filter_shape = [filter_size, embedding_size, 1, num_filters]
        W = tf.Variable(tf.truncated_normal(filter_shape, stddev=0.1), name="W")
        b = tf.Variable(tf.constant(0.1, shape=[num_filters]), name="b")
        conv = tf.nn.conv2d(
                            input_x_expanded,
                            W,
                            strides=[1, 1, 1, 1],
                            padding="VALID",
                            name="conv")
        h = tf.nn.tanh(tf.nn.bias_add(conv, b), name="activate")
        pooled = tf.nn.max_pool(
                            h,
                            ksize=[1, TITLE_LENGTH - filter_size + 1, 1, 1],
                            strides=[1, 1, 1, 1],
                            padding='VALID',
                            name="pool")
        pooled_outputs.append(pooled)

num_filters_total = num_filters * len(filter_sizes)
h_pool = tf.concat(pooled_outputs, 3)  #Tensor("concat_1:0", shape=(?, 1, 1, 300), dtype=float32)
h_pool_flat = tf.reshape(h_pool, [-1, num_filters_total])  #Tensor("Reshape:0", shape=(?, 300), dtype=float32)

# Final (unnormalized) scores and predictions
with tf.name_scope("output"):
    W = tf.get_variable(
        "W",
        shape=[num_filters_total, NUM_CLASSES],
        initializer=tf.contrib.layers.xavier_initializer())
    b = tf.Variable(tf.constant(0.1, shape=[NUM_CLASSES]), name="b")
    l2_loss += tf.nn.l2_loss(W)
    l2_loss += tf.nn.l2_loss(b)
    scores = tf.nn.xw_plus_b(h_pool_flat, W, b, name="scores")
    predictions = tf.argmax(scores, 1, name="predictions")

# # CalculateMean cross-entropy loss
with tf.name_scope("loss"):
    losses = tf.nn.softmax_cross_entropy_with_logits(logits=scores, labels=input_y)
    loss = tf.reduce_mean(losses) + l2_reg_lambda * l2_loss
    l2_loss += tf.nn.l2_loss(W)
    l2_loss += tf.nn.l2_loss(b)
    tf.summary.scalar('loss', loss)

with tf.name_scope("optimizing"):
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)
    
# # Accuracy
with tf.name_scope("accuracy"):
    correct_predictions = tf.equal(predictions, tf.argmax(input_y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_predictions, "float"), name="accuracy")
    tf.summary.scalar('accuracy', accuracy)

merged = tf.summary.merge_all()

In [ ]:
batch_size = 128
init = tf.global_variables_initializer()

index = 0
with tf.Session() as sess:
    train_writer = tf.summary.FileWriter('tensorboard/train',sess.graph)
    sess.run(init)
    for step in range(100):
        for start, end in zip(range(0, len(title_noun_vector_arr), batch_size), range(batch_size, len(title_noun_vector_arr), batch_size)):
            cost, acc, tb_summary, h_pool, _ = sess.run([loss, accuracy, merged, h_pool_flat, optimizer], feed_dict={input_x: title_noun_vector_arr[start:end], input_y: ctr_class_arr[start:end]})
            if index % 100 == 0:
                train_writer.add_summary(tb_summary, index)
                print('index:',index)
                print('cost:',cost)
                print('acc:',acc)
                print('---------')
            index += 1
        print('============')
        print('step:',step)
        print('cost:',cost)
        print('acc:',acc)
        print('h_pool:',h_pool)
        print('============')
        
    
    
    

index: 0
cost: 1.29363
acc: 0.28125
---------
